In [1]:
from joblib import Parallel, delayed

In [2]:
import ARX_Optimization as ID
import pandas
import numpy as np
import xlsxwriter
import pathlib
import configparser
from config import datadir

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
datadir = pathlib.Path(config['paths']['datadir']).expanduser()
inputs = pandas.read_csv(datadir/'Step1.csv')
# results.to_csv(datadir/'results.csv') !!NB

In [4]:
ydata1 = inputs.T1
udata1 = inputs.Q1
time1 = inputs.Time

In [5]:
# specify the order:
m, n = 2, 2

# initialize bounds:
bnds = ((-1,1),(-1,1),(-1,1),(-1,1))

# # Adding 10 degrees of noise:
# otpts, std_dev = ID.add_noise(ydata1, 5)

# # Changing the shape of the data in order to simply write it into the excel file later.
# rewrite_noise_data = []
# for j in range(0, len(ydata1)):
#     rewrite = [item[j] for item in otpts]
#     rewrite_noise_data.append(rewrite) 

def parallel_runs(N):
    
    # Adding 10 degrees of noise:
    otpts, std_dev = ID.add_noise(ydata1, 5)

    # Now to get the runtimes, coefficients and errors of the data:
    coefficients = []
    rts = []
    for i in range(0,N):
        ni = otpts[i,:]
        runtimes, coeffs = ID.DE_rt_coeffs(bnds, ni, udata1, m, n)
        coefficients.append(coeffs)
        rts.append(runtimes)
    return otpts, std_dev, coefficients, rts


In [6]:
# The for loop that performs identification is run in parallel ten times to obtain possible trends. 
block = Parallel(-1)(delayed(parallel_runs)(10) for i in range(0, 10))

In [7]:
block[0]

(array([[20.9495    , 20.9495    , 20.9495    , ..., 50.9234    ,
         50.9234    , 50.9234    ],
        [21.9495    , 21.9495    , 21.9495    , ..., 51.9234    ,
         51.9234    , 51.9234    ],
        [21.53846175, 22.30863244, 20.74105555, ..., 52.07159884,
         51.58825343, 51.87054204],
        ...,
        [20.56327585, 27.63524714, 27.08873126, ..., 48.7366683 ,
         53.64667336, 52.99304641],
        [19.40006765, 23.45617799, 22.06144175, ..., 41.86078144,
         45.73024561, 50.39161318],
        [29.02603433, 24.81298276, 27.61154015, ..., 53.13640739,
         44.210507  , 51.19729829]]),
 array([0.        , 0.55555556, 1.11111111, 1.66666667, 2.22222222,
        2.77777778, 3.33333333, 3.88888889, 4.44444444, 5.        ]),
 [array([ 5.06605712e-01,  4.85121112e-01, -5.31958324e-08,  5.05755292e-03]),
  array([5.06611823e-01, 4.85115023e-01, 3.26376157e-07, 5.05715385e-03]),
  array([ 0.47932831,  0.50964297, -0.00553717,  0.01206011]),
  array([ 0.505984

In [31]:
workbook = xlsxwriter.Workbook('C:/Users/annam/Desktop/Johanna/CSC411/Comparison-of-Identification-Techniques/Data/TCL_ARX_order2_DIFEV.xlsx')
worksheet1 = workbook.add_worksheet('Data')
worksheet2 = workbook.add_worksheet('Results')

bold = workbook.add_format({'bold': True})
col = 0
colcount = 0
modcount = 0
colct = 0
runcount = 0
row1=1 
for run in block:
    data = run[0]
    std_dev = run[1]
    coefficients = run[2]
    runtimes = run[3]
    for dataset in data:
        worksheet1.write(0, col, "run_"+str(colcount), bold)
        row = 1
        for value in dataset:
            worksheet1.write(row, col, value)
            row+=1
        col += 1
        modcount+=1
        if modcount %11 == 0:
            colcount +=1
            
    for sig, rt, coeff in zip(std_dev, runtimes, coefficients):
        worksheet2.write(row1, 0, "run_"+str(runcount))
        col1=1
        ct1=1
        ct2=1
        for cf in coeff:
            worksheet2.write(0, col1, "Coefficients"+str(colct), bold)
            if col1 <= m:
                worksheet2.write(0, col1, 'a'+str(ct1), bold)
                ct1 +=1
            else:
                worksheet2.write(0, col1, 'b'+str(ct2), bold)
                ct2 +=1
            
            worksheet2.write(row1, col1, cf)
            col1+=1
        worksheet2.write(0, col1, "std_dev", bold)
        worksheet2.write(0, col1+1, "Runtime"+str(colct), bold)
        worksheet2.write(row1, col1, sig)
        worksheet2.write(row1, col1+1, rt)
        row1+=1
    runcount+=1

workbook.close()
